In [1]:
import numpy as np


def compute_f(A, B, observed):
    n_nodes = len(observed)
    n_states = A.shape[0]
    f = np.zeros((n_nodes - 1, n_states, n_states))

    for i in range(n_nodes - 1):
        tmp = np.zeros((n_states, n_states))
        for j in range(n_states):
            for k in range(n_states):
                if B[k, observed[i + 1]] == 0:
                    tmp_B = np.min(B[B != 0])
                else:
                    tmp_B = np.log(B[k, observed[i + 1]])

                tmp[j, k] = np.log(A[j, k]) + tmp_B
        f[i] = tmp

    return f

In [2]:
A = np.array([[0.3, 0.7], [0.9, 0.1]])
B = np.array([[0.2, 0.8], [0.5, 0.5]])
observed = np.array([0, 1, 1])
compute_f(A, B, observed)

array([[[-1.42711636, -1.04982212],
        [-0.32850407, -2.99573227]],

       [[-1.42711636, -1.04982212],
        [-0.32850407, -2.99573227]]])

In [3]:
def Viterbi(A, B, observed):
    n_nodes = len(observed)
    n_states = A.shape[0]

    pmax = np.zeros((n_nodes - 1, n_states))
    phi = np.zeros((n_nodes - 1, n_states))

    f = compute_f(A, B, observed)

    pmax[0] = np.max((f[0]), axis=0)
    phi[0] = np.argmax(f[0], axis=0)

    for i in range(1, n_nodes - 1):
        tmp = ((f[i]).T + pmax[i - 1]).T

        pmax[i] = np.max(tmp, axis=0)

        phi[i] = np.argmax(tmp, axis=0)

    return pmax, phi

In [7]:
pmax, phi = Viterbi(A, B, observed)

In [17]:
print("pmax:\n", pmax, "\nphi\n", phi)

pmax:
 [[-0.32850407 -1.04982212]
 [-1.37832619 -1.37832619]] 
phi
 [[1. 0.]
 [1. 0.]]


In [25]:
first = np.argmax(pmax[-1])
a = int(phi[-1, first])
b = int(phi[-2, a])

In [26]:
print(first, a, b)

0 1 0


In [46]:
def reconstruct(pmax, phi):
    """
    Ricostruisce tutti i most probable, per ora fa schifo ma sembra funzionare, da ricontrollare
    """
    reconstruction = np.empty(len(phi)+1)

    curr = np.argmax(pmax[-1])
    reconstruction[-1] = curr

    for i in range(len(phi)-1, -1, -1):
        curr = int(phi[i, curr])
        reconstruction[i] = curr
    
    return reconstruction

In [47]:
reconstruct(pmax, phi)

array([0., 1., 0.])

### Experiement with matrix like the one we have (last row of spaces has 1)

In [60]:
A = np.array([[0.1, 0.8, 0.1],
              [0.8, 0.1, 0.1],
              [0.5, 0.5, 0]])

B = np.array([[0.8, 0.2, 0],
              [0.2, 0.8, 0],
              [0, 0, 1]])

observed = np.array([0, 2, 1])

In [68]:
def compute_f(A, B, observed):
    """
    Careful when handling
    - f0 is the first message (from first factor to node, it is just a vector) 
    - f contains all other factors evaluated
    """
    pi = A[-1]
    n_nodes = len(observed)
    n_states = A.shape[0]
    f = np.zeros((n_nodes-1, n_states, n_states))
    
    tmp = np.zeros((n_states, 1))
    for k in range(n_states):
        tmp[k] = pi[k] * B[k, observed[0]]
    
    f0 = tmp

    for i in range(1, n_nodes):
        tmp = np.zeros((n_states, n_states))
        
        for j in range(n_states): # over z1
            
            for k in range(n_states): # over z2
                tmp[j, k] = A[j, k] * B[k, observed[i]]
        
        f[i-1] = tmp
    

    return f0, f

In [83]:
f0, f = compute_f(A, B, observed)
print("f0:\n", f0, "\nf:\n", f)

f0:
 [[0.4]
 [0.1]
 [0. ]] 
f:
 [[[0.   0.   0.1 ]
  [0.   0.   0.1 ]
  [0.   0.   0.  ]]

 [[0.02 0.64 0.  ]
  [0.16 0.08 0.  ]
  [0.1  0.4  0.  ]]]


In [89]:
def Viterbi(f0, f):
    n_nodes = f.shape[0] + 1
    n_states = f.shape[1]

    pmax = np.zeros((n_nodes, n_states))    # Need one for every node
    phi = np.zeros((n_nodes - 1, n_states)) # Need one for every node other than the first one (no need to reconstruct it)

    pmax[0] = f0.flatten()

    for i in range(1, n_nodes):
        print(i)
        tmp = ((f[i-1]).T * pmax[i - 1]).T

        pmax[i] = np.max(tmp, axis=0) # by column

        phi[i-1] = np.argmax(tmp, axis=0) # i-1 cause this contains the reconstruction about the (i-1)th element

    return pmax, phi

In [90]:
Viterbi(f0, f)

1
2


(array([[0.4  , 0.1  , 0.   ],
        [0.   , 0.   , 0.04 ],
        [0.004, 0.016, 0.   ]]),
 array([[0., 0., 0.],
        [2., 2., 0.]]))

In [71]:
def compute_f_log(A, B, observed):
    """
    Careful when handling
    - f0 is the first message (from first factor to node, it is just a vector) 
    - f contains all other factors evaluated
    """
    pi = A[-1]
    n_nodes = len(observed)
    n_states = A.shape[0]
    f = np.zeros((n_nodes-1, n_states, n_states))
    
    tmp = np.zeros((n_states, 1))
    for k in range(n_states):
        tmp[k] = np.log(pi[k]) + np.log(B[k, observed[0]])
    
    f0 = tmp

    for i in range(1, n_nodes):
        tmp = np.zeros((n_states, n_states))
        
        for j in range(n_states): # over z1
            
            for k in range(n_states): # over z2
                tmp[j, k] = np.log(A[j, k]) + np.log(B[k, observed[i]])
        
        f[i-1] = tmp
    

    return f0, f

In [51]:
A = np.array([[0.1, 0.8, 0.1],
              [0.8, 0.1, 0.1],
              [0.5, 0.5, 0]])

B = np.array([[0.5, 0.5, 0],
              [0.5, 0.5, 0],
              [0, 0, 1]])

observed = np.array([0, 2, 1])

In [52]:
f = compute_f(A, B, observed)
print(f)

[[[0.05 0.4  0.  ]
  [0.4  0.05 0.  ]
  [0.25 0.25 0.  ]]

 [[0.   0.   0.1 ]
  [0.   0.   0.1 ]
  [0.   0.   0.  ]]

 [[0.05 0.4  0.  ]
  [0.4  0.05 0.  ]
  [0.25 0.25 0.  ]]]


- Errore in compute_f, parte dalla prima osservazione e invece dovrebbe partire dalla seconda (la prima non contribuisce ai messaggi se non )
- Controlla come viene considerato lo spazio

In [166]:
np.log(0.3) + np.log(0.4)

-2.120263536200091

In [ ]:
np.log(0.3) + np.log(0.4)

In [167]:
2*np.log(0.1) + np.log(0.4)

-5.521460917862246